# DPRPy 2022/2023
# HOMEWORK ASSIGNMENT NO. 2
## Szymon Ślusarz

## 1. Configuration
### 1.1 Importing necessary libraries and files

In [1]:
import numpy as np
import pandas as pd
from szymon_slusarz_assignment_2 import *

### 1.2 Reading the data
Before doing the main part of the exercise, I have to read all necessary data, to work on. I did it using __read_csv()__ command, from the _pandas_ library, and assigned the outputs to the corresponding names.

In [2]:
Posts = pd.read_csv("data/Posts.csv")
Comments = pd.read_csv("data/Comments.csv")
Badges = pd.read_csv("data/Badges.csv")
Votes = pd.read_csv("data/Votes.csv")
Users = pd.read_csv("data/Users.csv")

### 1.3 Reference solutions
To check if our solutions are correct, we have to create reference solutions, using __read_sql_query()__ command from _pandas_ library, which returns output of a SQL query given as a parameter.

In [3]:
import os, os.path
import sqlite3
import tempfile

baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

query1 = pd.read_sql_query("""SELECT STRFTIME('%Y', CreationDate) AS Year, COUNT(*) AS TotalNumber
FROM Posts
GROUP BY Year""", conn)

query2 = pd.read_sql_query("""SELECT Id, DisplayName, SUM(ViewCount) AS TotalViews
FROM Users
JOIN (
SELECT OwnerUserId, ViewCount FROM Posts WHERE PostTypeId = 1
) AS Questions
ON Users.Id = Questions.OwnerUserId
GROUP BY Id
ORDER BY TotalViews DESC
LIMIT 10""", conn)

query3 = pd.read_sql_query("""SELECT Year, Name, MAX((Count * 1.0) / CountTotal) AS MaxPercentage
FROM (
SELECT BadgesNames.Year, BadgesNames.Name, BadgesNames.Count, BadgesYearly.CountTotal
FROM (
SELECT Name, COUNT(*) AS Count, STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
GROUP BY Name, Year
) AS BadgesNames
JOIN (
SELECT COUNT(*) AS CountTotal, STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
GROUP BY YEAR
) AS BadgesYearly
ON BadgesNames.Year = BadgesYearly.Year
)
GROUP BY Year""", conn)

query4 = pd.read_sql_query("""SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10""", conn)

query5 = pd.read_sql_query("""SELECT Posts.Title, STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date, VotesByAge.*
FROM Posts
JOIN (
SELECT PostId,
MAX(CASE WHEN VoteDate = 'before' THEN Total ELSE 0 END) BeforeCOVIDVotes,
MAX(CASE WHEN VoteDate = 'during' THEN Total ELSE 0 END) DuringCOVIDVotes,
MAX(CASE WHEN VoteDate = 'after' THEN Total ELSE 0 END) AfterCOVIDVotes,
SUM(Total) AS Votes
FROM (
SELECT PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2022' THEN 'after'
WHEN '2021' THEN 'during'
WHEN '2020' THEN 'during'
WHEN '2019' THEN 'during'
ELSE 'before'
END VoteDate, COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId IN (3, 4, 12)
GROUP BY PostId, VoteDate
) AS VotesDates
GROUP BY VotesDates.PostId
) AS VotesByAge ON Posts.Id = VotesByAge.PostId
WHERE Title NOT IN ('') AND DuringCOVIDVotes > 0
ORDER BY DuringCOVIDVotes DESC, Votes DESC
LIMIT 20""", conn)

conn.close()

## 2. Introduction
This project's goal, is to learn how to process data in python, using _pandas_ library. We were given five SQL queries and the task was to convert them using commands and methods from the mentioned library.

## 3. QUERIES
In this section, I will execute each of my functions from _szymon_slusarz_assignment_2.py_, and then compare it to the reference solutions from the 1.3 section. I will do this using __.equal()__ function from the _pandas_ package, which returns _TRUE_, if the compared outputs are equal.

### 3.1 QUERY #1
This query is showing total number of posts posted in each year.

#### Results presentation

Output of _solution_1(Posts)_ and _query1_:

In [4]:
solution_1(Posts)

,Year,TotalNumber
0,2011,4334
1,2012,6335
2,2013,7614
3,2014,10953
4,2015,14007
5,2016,17087
6,2017,16154
7,2018,15141
8,2019,15499
9,2020,6203


In [5]:
query1

,Year,TotalNumber
0,2011,4334
1,2012,6335
2,2013,7614
3,2014,10953
4,2015,14007
5,2016,17087
6,2017,16154
7,2018,15141
8,2019,15499
9,2020,6203


#### Final comparison

In [6]:
solution_1(Posts).equals(query1)

True

As we can see __.equals()__ command returned true, so both solutions are equivalent.

### 3.2 QUERY #2
This query is showing 10 users, who asked the most viewed questions.

#### Results presentation

Output of _solution_2(Posts, Users)_ and _query2_:

In [7]:
solution_2(Posts, Users)

,Id,DisplayName,TotalViews
0,101,Mark Mayo,2217642.0
1,9009,JonathanReez,2187770.0
2,693,RoflcoptrException,1929754.0
3,140,hippietrail,1927767.0
4,26,Gagravarr,1607181.0
5,3736,nsn,1241921.0
6,396,jrdioko,1016211.0
7,10051,Nean Der Thal,960761.0
8,1893,lambshaanxy,896633.0
9,2509,gerrit,868851.0


In [8]:
query2

,Id,DisplayName,TotalViews
0,101,Mark Mayo,2217642.0
1,9009,JonathanReez,2187770.0
2,693,RoflcoptrException,1929754.0
3,140,hippietrail,1927767.0
4,26,Gagravarr,1607181.0
5,3736,nsn,1241921.0
6,396,jrdioko,1016211.0
7,10051,Nean Der Thal,960761.0
8,1893,lambshaanxy,896633.0
9,2509,gerrit,868851.0


#### Final comparison

In [9]:
solution_2(Posts, Users).equals(query2)

True

As we can see __.equals()__ command returned true, so both solutions are equivalent.

### 3.3 QUERY #3
This query is showing the badge name, with the highest ratio (percentage) to all badges in a given year.

#### Results presentation

Output of _solution_3(Badges)_ and _query3_:

In [10]:
solution_3(Badges)

,Year,Name,MaxPercentage
0,2011,Supporter,0.166925
1,2012,Supporter,0.130064
2,2013,Supporter,0.117536
3,2014,Autobiographer,0.214061
4,2015,Autobiographer,0.147789
5,2016,Student,0.119992
6,2017,Student,0.136280
7,2018,Student,0.133628
8,2019,Student,0.137143
9,2020,Yearling,0.118042


In [11]:
query3

,Year,Name,MaxPercentage
0,2011,Supporter,0.166925
1,2012,Supporter,0.130064
2,2013,Supporter,0.117536
3,2014,Autobiographer,0.214061
4,2015,Autobiographer,0.147789
5,2016,Student,0.119992
6,2017,Student,0.136280
7,2018,Student,0.133628
8,2019,Student,0.137143
9,2020,Yearling,0.118042


#### Final comparison

In [12]:
solution_3(Badges).equals(query3)

True

As we can see __.equals()__ command returned true, so both solutions are equivalent.

### 3.4 QUERY #4
This query is showing 10 posts statistics with the highest number of comments score. The statistics include post title, amount of comments, how many times it was viewed, mentioned total comments score, author name, reputation and location.

#### Results presentation

Output of _solution_4(Posts, Users, Comments)_ and _query4_:

In [13]:
solution_4(Posts, Users, Comments)

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,61309.0,630,Megha,1679,NaN
1,How to intentionally get denied entry to the U...,37,72856.0,618,user11743,1997,NaN
2,Can I wear a bulletproof vest while traveling ...,26,23467.0,403,Ulkoma,9190,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,113001.0,388,hippietrail,75994,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,14308.0,358,Emma-louise,611,NaN
5,Immigration officer that stopped me at the air...,24,22495.0,345,DiegoJ,1273,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,29344.0,343,Silver Dragon,943,"San Francisco, United States"
7,Can I fly with a gold bar?,25,55450.0,333,Ulkoma,9190,"London, United Kingdom"
8,How to book a flight if my passport doesn't st...,23,18029.0,330,kukis,894,"Sztokholm, Szwecja"
9,Why don't airlines have backup planes just in ...,26,18809.0,328,clickbait,542,NaN


In [14]:
query4

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,61309.0,630,Megha,1679,None
1,How to intentionally get denied entry to the U...,37,72856.0,618,user11743,1997,None
2,Can I wear a bulletproof vest while traveling ...,26,23467.0,403,Ulkoma,9190,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,113001.0,388,hippietrail,75994,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,14308.0,358,Emma-louise,611,None
5,Immigration officer that stopped me at the air...,24,22495.0,345,DiegoJ,1273,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,29344.0,343,Silver Dragon,943,"San Francisco, United States"
7,Can I fly with a gold bar?,25,55450.0,333,Ulkoma,9190,"London, United Kingdom"
8,How to book a flight if my passport doesn't st...,23,18029.0,330,kukis,894,"Sztokholm, Szwecja"
9,Why don't airlines have backup planes just in ...,26,18809.0,328,clickbait,542,None


#### Final comparison

In [15]:
solution_4(Posts, Users, Comments).equals(query4)

True

As we can see __.equals()__ command returned true, so both solutions are equivalent.

### 3.5 QUERY #5
To present the most intuitive interpretation of this query, it is a good idea to divide it into two things.

First of all we are operating on votes which type is “DownMod”, “Offensive” or “Spam”, so rather the negative ones. We are counting the number of mentioned votes in each of newly, based by year, created group. These groups are: after COVID votes (those created in 2022), during COVID votes (those created in 2019-2021) and before COVID votes (those created before 2019).

The final output shows the data frame with 20 most voted posts during COVID pandemics, excluding these without a title or with less votes during COVID than 1. It shows statistics such as title, post creation date, post ID, number of votes before/during/after COVID, and the total amount of votes.

#### Results presentation

Output of _solution_5(Posts, Votes)_ and _query5_:

In [16]:
solution_5(Posts, Votes)

,Title,Date,PostId,BeforeCOVIDVotes,DuringCOVIDVotes,AfterCOVIDVotes,Votes
0,Which countries have still NOT yet reported a ...,2020-03-15,154995,0,23,0,23
1,Why don't hotels offer (at least) 1 kitchen bo...,2019-02-17,132343,0,17,0,17
2,Where can I go to avoid planes overhead?,2019-04-30,137619,0,13,0,13
3,Can male guests ejaculate in hotels' dispenser...,2019-08-29,144746,0,13,0,13
4,Can I make a leisure trip to California (vacat...,2021-01-18,162140,0,13,0,13
5,Why do some countries have inconsistent rules ...,2021-09-30,168959,0,13,0,13
6,What are one's options when facing religious d...,2019-09-24,147417,0,12,0,12
7,"How can you escape, if racists in London, who ...",2020-03-04,154601,0,12,0,12
8,Where's this yellowish building in Texas?,2020-05-21,157337,0,12,0,12
9,How long before boarding gates close are last ...,2021-08-25,168374,0,12,0,12


In [17]:
query5

,Title,Date,PostId,BeforeCOVIDVotes,DuringCOVIDVotes,AfterCOVIDVotes,Votes
0,Which countries have still NOT yet reported a ...,2020-03-15,154995,0,23,0,23
1,Why don't hotels offer (at least) 1 kitchen bo...,2019-02-17,132343,0,17,0,17
2,Where can I go to avoid planes overhead?,2019-04-30,137619,0,13,0,13
3,Can male guests ejaculate in hotels' dispenser...,2019-08-29,144746,0,13,0,13
4,Can I make a leisure trip to California (vacat...,2021-01-18,162140,0,13,0,13
5,Why do some countries have inconsistent rules ...,2021-09-30,168959,0,13,0,13
6,What are one's options when facing religious d...,2019-09-24,147417,0,12,0,12
7,"How can you escape, if racists in London, who ...",2020-03-04,154601,0,12,0,12
8,Where's this yellowish building in Texas?,2020-05-21,157337,0,12,0,12
9,How long before boarding gates close are last ...,2021-08-25,168374,0,12,0,12


#### Final comparison

In [18]:
solution_5(Posts, Votes).equals(query5)

True

As we can see __.equals()__ command returned true, so both solutions are equivalent.